In [ ]:
from optimisation_models.prosumer_rho_model import DataPreparation, OptimisationParameters, ProsumerRho, ProcessOptimisedData

In [ ]:
solar_home_file='data/raw/solar_home_processed_in_sample.csv'
price_file = 'data/predispatch_prices/PD_NSW_2021.csv'
months = range(1, 13) # Month order can be changed
data_preparation = DataPreparation(solar_home_file, price_file)

In [ ]:
for optimised_month in months:
    df_prosumer = data_preparation.get_prosumer_data(month=optimised_month)
    df_price = data_preparation.get_pd_price_data(month=optimised_month)

    optimisation_parameters = OptimisationParameters(df_prosumer, df_price)
    rolling_model = ProsumerRho(optimisation_parameters)

    # Run this to get the prosumer rho
    rolling_model.rolling_optimisation(n_receding_horizons=2)

    folder_name = f'data/processed/prosumer_rho/month_{optimised_month}'
    rolling_model.save_rolling_data(folder_name)

In [ ]:
process_optimised_data = ProcessOptimisedData()
for optimised_month in months:
    dfs_agg_rolling = process_optimised_data.compile_aggregate_data(f'data/processed/prosumer_rho/month_{optimised_month}')
    agg_folder_name = f'data/processed/prosumer_rho/aggregate_data/month_{optimised_month}'
    process_optimised_data.save_aggregate_rolling_data(agg_folder_name)

In [ ]:
process_optimised_data = ProcessOptimisedData()
dfs_agg_binding = []
lookahed_interval = 0
for optimised_month in months:
    df_agg_binding = process_optimised_data.get_lookahead_values(f'data/processed/prosumer_rho/aggregate_data/month_{optimised_month}', lookahed_interval)
    dfs_agg_binding.append(df_agg_binding)
agg_binding_folder_name = f'data/processed/prosumer_rho/aggregate_data'
process_optimised_data.save_lookahead_data(dfs_agg_binding, lookahed_interval, agg_binding_folder_name)